In [2]:
import pandas as pd
import numpy as np

# To remove the scientific notation from numpy arrays
np.set_printoptions(suppress=True)

CarPrices=pd.read_pickle('CarPricesData.pkl')
CarPrices.head()

,Age,KM,Weight,HP,MetColor,CC,Doors,Price
0,23.0,46986,1165.0,90,1,2000.0,3,13500
1,23.0,72937,1165.0,90,1,2000.0,3,13750
2,24.0,41711,1165.0,90,1,2000.0,3,13950
3,26.0,48000,1165.0,90,0,2000.0,3,14950
4,30.0,38500,1170.0,90,0,2000.0,3,13750


In [3]:
Targetvariable = ['Price']
Predictors=['Age', 'KM', 'Weight', 'HP', 'MetColor', 'CC', 'Doors']

X = CarPrices[Predictors].values
y=CarPrices[Targetvariable].values

### Sandardization of data ###
from sklearn.preprocessing import StandardScaler
PredictorScaler=StandardScaler()
TargetVarScaler=StandardScaler()

# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X)
TargetVarScalerFit=TargetVarScaler.fit(y)


X = PredictorScalerFit.transform(X)
Y = TargetVarScalerFit.transform(y)

# Split the data into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


# Quick sanity check with the shapes of Training and testing datasets
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1004, 7)
(1004, 1)
(431, 7)
(431, 1)


In [4]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [4]:
#creating the ANN model
model = Sequential()


In [5]:
# Defining the Input layer and FIRST hidden layer, both are same!
model.add(Dense(units=5,input_dim = 7,kernel_initializer='normal',activation='relu'))

In [6]:
model.add(Dense(units=5,kernel_initializer='normal',activation = 'tanh'))

In [7]:
# Since we will be predicting a single number
model.add(Dense(1,kernel_initializer='normal'))

In [8]:
# Compiling the model
model.compile(loss ='mean_squared_error',optimizer = 'adam')

In [9]:
model.fit(X_train,y_train,batch_size=20,epochs=50,verbose=1)

Epoch 1/50
51/51 [==============================] - 1s 1ms/step - loss: 127444280.0000
Epoch 2/50
51/51 [==============================] - 0s 1ms/step - loss: 127441688.0000
Epoch 3/50
51/51 [==============================] - 0s 1ms/step - loss: 127435272.0000
Epoch 4/50
51/51 [==============================] - 0s 3ms/step - loss: 127423696.0000
Epoch 5/50
51/51 [==============================] - 0s 2ms/step - loss: 127409744.0000
Epoch 6/50
51/51 [==============================] - 0s 1ms/step - loss: 127396600.0000
Epoch 7/50
51/51 [==============================] - 0s 1ms/step - loss: 127385104.0000
Epoch 8/50
51/51 [==============================] - 0s 1ms/step - loss: 127374872.0000
Epoch 9/50
51/51 [==============================] - 0s 1ms/step - loss: 127365496.0000
Epoch 10/50
51/51 [==============================] - 0s 1ms/step - loss: 127356568.0000
Epoch 11/50
51/51 [==============================] - 0s 1ms/step - loss: 127347968.0000
Epoch 12/50
51/51 [======================

In [10]:
#Finding best set of parameters using manual grid search

def functionfindbestparams(X_train,y_train,X_test,y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[5, 10, 15, 20]
    epoch_list  =   [5, 10, 50, 100]
    
    import pandas as pd
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'Accuracy'])
    
    
    #initializing the Trails
    Trailnumber = 0
    for batch_size_trail in batch_size_list:
        for epochs_trail in epoch_list:
          Trailnumber+=1
          
          #creating the ANN
          model = Sequential()
          #First layer
          model.add(Dense(units=5, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
          
          #secod layer
          model.add(Dense(units = 5,kernel_initializer='normal', activation='relu'))
          
          model.add(Dense(1, kernel_initializer='normal'))
          
          # Compiling the model
          model.compile(loss='mean_squared_error', optimizer='adam')
          
          # Fitting the ANN to the Training set
          model.fit(X_train, y_train ,batch_size = batch_size_trail, epochs = epochs_trail, verbose=0)
          
          
          MAPE = np.mean(100*(np.abs(y_test-model.predict(X_test))/y_test))
          
          # printing the results of the current iteration
          print(Trailnumber, 'Parameters:','batch_size:', batch_size_trail,'-', 'epochs:',epochs_trail, 'Accuracy:', 100-MAPE)
            
          SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[Trailnumber, str(batch_size_trail)+'-'+str(epochs_trail), 100-MAPE]],
                                                                    columns=['TrialNumber', 'Parameters', 'Accuracy'] ))
    return(SearchResultsData)
 
  

In [11]:
# Calling the function
Results_Data = functionfindbestparams(X_train,y_train,X_test,y_test)


14/14 [==============================] - 0s 920us/step
1 Parameters: batch_size: 5 - epochs: 5 Accuracy: 1.0338468426366205


C:\Users\amits\AppData\Local\Temp\ipykernel_22236\1652241918.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[Trailnumber, str(batch_size_trail)+'-'+str(epochs_trail), 100-MAPE]],


14/14 [==============================] - 0s 1ms/step
2 Parameters: batch_size: 5 - epochs: 10 Accuracy: 6.623781398385191


C:\Users\amits\AppData\Local\Temp\ipykernel_22236\1652241918.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[Trailnumber, str(batch_size_trail)+'-'+str(epochs_trail), 100-MAPE]],


14/14 [==============================] - 0s 1ms/step
3 Parameters: batch_size: 5 - epochs: 50 Accuracy: 87.28643608831668


...

C:\Users\amits\AppData\Local\Temp\ipykernel_22236\1652241918.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[Trailnumber, str(batch_size_trail)+'-'+str(epochs_trail), 100-MAPE]],


14/14 [==============================] - 0s 2ms/step
15 Parameters: batch_size: 20 - epochs: 50 Accuracy: 4.34031402748775


C:\Users\amits\AppData\Local\Temp\ipykernel_22236\1652241918.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[Trailnumber, str(batch_size_trail)+'-'+str(epochs_trail), 100-MAPE]],


14/14 [==============================] - 0s 2ms/step
16 Parameters: batch_size: 20 - epochs: 100 Accuracy: 22.865297268341223


C:\Users\amits\AppData\Local\Temp\ipykernel_22236\1652241918.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[Trailnumber, str(batch_size_trail)+'-'+str(epochs_trail), 100-MAPE]],


In [24]:
#Training the ANN model with the best parameters

model.fit(X_train, y_train ,batch_size = 20, epochs = 100, verbose=0)


predictions=model.predict(X_test)

predictions = TargetVarScalerFit.inverse_transform(predictions)

#scaling the y_test

y_test_org = TargetVarScalerFit.inverse_transform(y_test)

Test_Data=PredictorScalerFit.inverse_transform(X_test)
 
TestingData=pd.DataFrame(data=Test_Data, columns=Predictors)
TestingData['Price']=y_test_org
TestingData['PredictedPrice']=predictions
TestingData.head()

14/14 [==============================] - 0s 920us/step


,Age,KM,Weight,HP,MetColor,CC,Doors,Price,PredictedPrice
0,59.0,80430.0,1065.0,110.0,1.0,1600.0,3.0,3.590510e+07,1060465.0
1,62.0,64797.0,1075.0,110.0,1.0,1600.0,5.0,2.885249e+07,1060465.0
2,59.0,130000.0,1135.0,72.0,1.0,2000.0,4.0,2.706679e+07,1060465.0
3,69.0,42800.0,1050.0,110.0,1.0,1600.0,3.0,3.590510e+07,1060465.0
4,65.0,47014.0,1015.0,86.0,1.0,1300.0,3.0,3.229763e+07,1060465.0
